In [ ]:
import requests
import json
import os
from dotenv import load_dotenv
import asyncio
from typing import Optional, Type, Any, Dict, Union, Tuple, List
from pydantic import BaseModel
import logging

load_dotenv()  # This loads the .env file into the environment

import openai

client = openai.AsyncOpenAI()

async def call_openai(
    messages,
    model_name = "o3-mini",
    output_type: Optional[Type[BaseModel]] = None,
    max_tokens: int = 1500,
    reasoning_effort = "medium",
    max_retries=3,
    tools: Optional[Dict[str, Any]] = None,
) -> Union[str, BaseModel]:
    """
    Calls the LLM using OpenAI's chat completion or HF, returning either raw text
    or a Pydantic-validated object.
    """

    last_exception = None
    for attempt in range(max_retries):
        try:
            if output_type is None:
                response = await client.chat.completions.create(
                    messages=messages,
                    model=model_name,
                    max_completion_tokens=max_tokens,
                    tools=tools
                )
                if tools:
                    print(response)
                    result = response.choices[0].message
                else:
                    result = response.choices[0].message.content
                # logging.info("\nRESPONSE:", result)
                # logging.info("=" * 80)
                return result
            else:
                response = await client.beta.chat.completions.parse(
                    messages=messages,
                    model=model_name,
                    max_completion_tokens=max_tokens,
                    response_format=output_type,
                )
                parsed_obj = response.choices[0].message.parsed
                result = output_type.model_validate(parsed_obj)
                # logging.info("\nRESPONSE:", result)
                # logging.info("=" * 80)
                return result

        except Exception as e:
            logging.info(
                f"[DEBUG] Attempt {attempt+1}/{max_retries} failed with exception: {e}"
            )
            logging.info("[DEBUG] User messages (verbatim):")
            for idx, msg in enumerate(messages):
                logging.info(f"  Message {idx+1} - role='{msg['role']}':")
                logging.info(msg["content"])
            last_exception = e
            await asyncio.sleep(1.0 * (attempt + 1))

    raise last_exception


await call_openai(messages=[
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What is the capital of the United States?"},
])

'The capital of the United States is Washington, D.C. This city serves as the center of the federal government, housing major institutions like the White House, the Capitol, and the Supreme Court.'

In [ ]:
propose_new_tool = {
    "name": "propose_new_tool",
    "description": "Propose the function specs for one or two new python functions you can call anytime as new tool calls later on. Brainstorm some functions that do specific algorithmic computations about the board that helps your decision, and write detailed specs of those.",
    "input_schema": {
        "type": "object",
        "properties": {
            "tool_name_1": {
                "type": "string",
                "description": "Name of the tool you will use"
            },
            "tool_name_2": {
                "type": "string",
                "description": "Name of the tool you will use"
            },
            "function_spec_1": {
                "type": "string",
                "description": "A detailed description of the function you want to propose, as well as the signature of its outputs. The function should only receive an input which is a 2D array representing the current visible board state, where in the array X represents a hit, O represents a miss, and ~ represents unknown cells."  
            },
            "function_spec_2": {
                "type": "string",
                "description": "A detailed description of the function you want to propose, as well as the signature of its outputs. The function should only receive an input which is a 2D array representing the current visible board state, where in the array X represents a hit, O represents a miss, and ~ represents unknown cells."  
            }
        },
        "required": ["tool_name_1", "function_spec_1"]
    }
}

In [ ]:
class Move(BaseModel):
    action: str
    desc: str
    score: Optional[float] = None


class MoveChoices(BaseModel):
    choices: List[Move]


class ChessGame:
    def __init__(self, env):
        self.env = env